# Part 1: Social Media Behaviour Data Analysis


---

### Install Python packages (pip only)

In [1]:
#e.g., %pip install some-package
%pip install networkx
%pip install scipy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Import Python packages

In [2]:
#e.g., import some-package
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

---

### Task 1 of 1

Examine the Graph Modelling Language (gml) files "socialmedia_cmt224_reply_network.gml" (reply network) and "socialmedia_cmt224_social_network.gml" (social network) which represent Twitter data between a sample of users over several days at the time of the Higgs boson particle discovery. Both networks are directed and share the same ids for nodes (anonymised Twitter users).  However, the shared user ids are contained within the "label" attribute in the .gml files, not the node "id" attribute of each individual .gml file.

In the reply network, an edge from a node, 𝑢, to some other node, 𝑣, indicates that 𝑢 replied to a Tweet made by 𝑣 during the time period. Replies are also Tweets. Edges are weighted with the weight representing the number of times this happened over the time period.

In the social network, an edge from node 𝑢 to 𝑣 indicates that 𝑢 follows 𝑣 on the social media platform.

Using these networks, answer the following questions:

##### Q1. What fraction of users do not reply to or follow any other user, but have had others reply to their Tweets?

In [3]:
#CODE:
# 1 加载数据集
rn = nx.read_gml("socialmedia_cmt224_reply_network.gml")
sn = nx.read_gml("socialmedia_cmt224_social_network.gml")

# 回复网络节点数、边数
print(f"Number of nodes: {rn.number_of_nodes()}")
print(f"Number of edges: {rn.number_of_edges()}")
# 社交网络节点数、边数
print(f"Number of nodes: {rn.number_of_nodes()}")
print(f"Number of edges: {rn.number_of_edges()}")

# 2 找出那些不回复别人、不关注别人，但是有其他人回复他们
# 也就是回复和社交网络中只进不出
r_order = list(rn.nodes())
s_order = list(sn.nodes())

only_replyed_by_others = []
for node in r_order:
    r_in_degree = rn.in_degree(node)
    r_out_degree = rn.out_degree(node)
    s_out_degree = sn.out_degree(node)
    if r_in_degree > 0 and r_out_degree == 0 and s_out_degree == 0:
        only_replyed_by_others.append(node)

print(len(only_replyed_by_others))
# 再除个回复网络所有节点

Number of nodes: 16737
Number of edges: 11430
Number of nodes: 16737
Number of edges: 11430
152


##### Q2. How does the topological structure of the reply network differ from the social network in terms of overall sparsity of edges between users and the number of connected groups of users?

In [4]:
#CODE: 
density_reply = nx.density(rn)
density_social = nx.density(sn)
rn_number_scc_components = nx.number_strongly_connected_components(rn)
rn_number_wcc_components = nx.number_weakly_connected_components(rn)
sn_number_scc_components = nx.number_strongly_connected_components(sn)
sn_number_wcc_components = nx.number_weakly_connected_components(sn)
print(f"The density of reply network: {density_reply}")
print(f"The density of social network: {density_social}")
print(f"Number of strongly connected components in reply network: {rn_number_scc_components}")
print(f"Number of weakly connected components in social network: {rn_number_wcc_components}")
print(f"Number of strongly connected components in reply network: {sn_number_scc_components}")
print(f"Number of weakly connected components in social network: {sn_number_wcc_components}")
# 只保留弱连通组件

The density of reply network: 4.080533494732535e-05
The density of social network: 0.000992358613762732
Number of strongly connected components in reply network: 16217
Number of weakly connected components in social network: 5920
Number of strongly connected components in reply network: 4648
Number of weakly connected components in social network: 436


##### Q3. Does the number of users a user follows in the social network correlate with the number of replies that they make?

In [5]:
#CODE:
# 计算回复、关注网络中每个节点的出度，然后分析两个列表的联系
# 提取社交网络中的出度（关注数）和回复网络中的出度（回复数）
r_out_degree_sequence = list(dict(rn.out_degree(r_order)).values())
s_out_degree_sequence = list(dict(sn.out_degree(s_order)).values())
r, p = stats.pearsonr(r_out_degree_sequence, s_out_degree_sequence)
print(f"pearson r: {r:.2f}, {p:.2f}")

pearson r: 0.01, 0.17


##### Q4. Is a user that replies to another user's Tweet multiple times more likely to follow that user in comparison to if they only replied once?

In [6]:
#CODE:
w_rn = nx.DiGraph(rn)

# 初始化计数器
reply_count = {}
follow_with_multireplies = 0
sum_multireplies = 0
follow_with_onereply = 0
sum_onereply = 0

# 分析回复与关注的关系
for u, v, data in w_rn.edges(data=True):
    weight = data.get('occurrences', 1)
    # 检查u是否关注了v
    if sn.has_edge(u, v):
        # 回复多次
        if weight > 1:
            follow_with_multireplies += 1
        # 回复单次
        else:
            follow_with_onereply += 1
    # 统计所有的多次回复
    if weight > 1:
        sum_multireplies += 1
    # 统计所有的单次回复
    else:
        sum_onereply += 1

prob_single_reply = follow_with_onereply / sum_onereply if sum_onereply else 0
prob_multi_replies = follow_with_multireplies / sum_multireplies if sum_multireplies else 0

# 输出结果
print(f"所有的单次回复数量: {sum_onereply}")
print(f"所有的多次回复数量: {sum_multireplies}")
print(f"用户单次回复并关注对方的概率: {prob_single_reply:.2f}")
print(f"用户多次回复并关注对方的概率: {prob_multi_replies:.2f}")

所有的单次回复数量: 8416
所有的多次回复数量: 3014
用户单次回复并关注对方的概率: 0.84
用户多次回复并关注对方的概率: 0.88


##### Q5. How many users have only mutual following connections (i.e., every user they follow also follows them) and only mutual reply connections with these same users?

In [7]:
#CODE:
def find_mutual_users(sn, rn):
    # 初始化一个列表来存储完全符合条件的用户
    sn_rn_mutual_users = []
    # 遍历社交网络中的所有用户
    for user in rn.nodes():
        # 获取当前用户的关注列表和被关注列表
        follow_out = set(sn.successors(user))
        follow_in = set(sn.predecessors(user))
        # 互相关注列表
        mutual_follows = follow_out & follow_in
        
        # 检查回复网络
        if mutual_follows:
            # 获取当前用户互相回复列表
            reply_out = set(rn.successors(user))
            reply_in = set(rn.predecessors(user))
            # 确保用户仅回复了其互相关注的人，且这些人也回复了他
            if reply_out == mutual_follows and reply_in == mutual_follows:
                sn_rn_mutual_users.append(user)
    return sn_rn_mutual_users
mutual_users = find_mutual_users(sn, rn)
print(len(mutual_users))

312
